# Update current density - Issue [#32](https://github.com/wd15/extremefill2D/issues/32)

## The Old
The old current density is 

$$
i = \frac{c_{\text{cu}}}{c_{\text{cu}}^{\infty}}
    \left(i_0 + i_1 \theta\right)
    \left(e^{-\alpha \bar{F} \eta} - e^{\left(2 - \alpha\right) \bar{F} \eta}\right)
$$

where $\bar{F} = \frac{F}{R T}$. The deriviative w.r.t $\eta$ is

$$
\frac{\partial i}{\partial \eta} = \bar{F}
    \frac{c_{\text{cu}}}{c_{\text{cu}}^{\infty}}
    \left(i_0 + i_1 \theta\right)
    \left(-\alpha e^{-\alpha \bar{F} \eta} - \left(2 - \alpha\right) e^{\left(2 - \alpha\right) \bar{F} \eta}\right)
$$

This is used to stablize the equation for potential.

The old code for this is (note that `self.potential` is $-\eta$) in the code),
```python
    def calc_dep_vars(self, params):
        Fbar = params.faradaysConstant / params.gasConstant / params.temperature
        self.coeff_forward = params.alpha * Fbar
        self.coeff_backward = (2 - params.alpha) * Fbar
        exp_forward = numerix.exp(self.coeff_forward * self.potential)
        exp_backward = numerix.exp(-self.coeff_backward * self.potential)
        I0 = (params.i0 + params.i1 * self.interfaceTheta)
        cbar =  self.cupric / params.bulkCupric
        self.beta_forward = cbar * I0 * exp_forward
        self.beta_backward = cbar * I0 * exp_backward
        self.baseCurrent = I0 * (exp_forward - exp_backward)
        self.currentDensity = cbar * self.baseCurrent
        self.currentDerivative = cbar * I0 * (self.coeff_forward *  exp_forward + self.coeff_backward * exp_backward)
        self.depositionRate = self.currentDensity * params.omega / params.charge / params.faradaysConstant
```

## The New

The new current density is

$$
i = \frac{c_{\text{cu}}}{c_{\text{cu}}^{\infty}}
    \left[
    i_0 \left(1 - \theta\right)
    \left(e^{-\alpha_0 \bar{F} \eta} - e^{\left(1 - \alpha_0 \right) \bar{F} \eta}\right)
    + 
    i_1 \theta
    \left(e^{-\alpha_1 \bar{F} \eta} - e^{\left(1 - \alpha_1 \right) \bar{F} \eta}\right)
    \right]
$$
and
$$
i = \bar{F}
    \frac{c_{\text{cu}}}{c_{\text{cu}}^{\infty}}
    \left[
    i_0 \left(1 - \theta\right)
    \left(-\alpha_0 e^{-\alpha_0 \bar{F} \eta} - \left(1 - \alpha_0 \right) e^{\left(1 - \alpha_0 \right) \bar{F} \eta}\right)
    + 
    i_1 \theta
    \left(-\alpha_1 e^{-\alpha_1 \bar{F} \eta} - \left(1 - \alpha_1 \right) e^{\left(1 - \alpha_1 \right) \bar{F} \eta}\right)
    \right]
$$

The new code is

```python
    def calc_dep_vars(self, params):

        Fbar = params.faradaysConstant / params.gasConstant / params.temperature
        self.coeff_forward0 = params.alpha0 * Fbar
        self.coeff_backward0 = (1 - params.alpha0) * Fbar
        self.coeff_forward1 = params.alpha1 * Fbar
        self.coeff_backward1 = (1 - params.alpha1) * Fbar
        exp_forward0 = numerix.exp(self.coeff_forward0 * self.potential)
        exp_backward0 = numerix.exp(-self.coeff_backward0 * self.potential)
        exp_forward1 = numerix.exp(self.coeff_forward1 * self.potential)
        exp_backward1 = numerix.exp(-self.coeff_backward1 * self.potential)
        cbar =  self.cupric / params.bulkCupric
        I0 = params.i0 * (1 - self.theta)
        I1 = params.i1 * self.theta
        self.beta_forward0 = cbar * I0 * exp_forward0
        self.beta_backward0 = cbar * I0 * exp_backward0
        self.beta_forward1 = cbar * I1 * exp_forward1
        self.beta_backward1 = cbar * I1 * exp_backward1

        self.baseCurrent = I0 * (exp_forward0 - exp_backward0) + I1 * (exp_forward1 - exp_backward1)
        self.currentDensity = cbar * self.baseCurrent
        self.currentDerivative = cbar * (I0 * (self.coeff_forward0 *  exp_forward0 + self.coeff_backward0 * exp_backward0) \
                                         + I1 * (self.coeff_forward1 *  exp_forward1 + self.coeff_backward1 * exp_backward1))
        self.depositionRate = self.currentDensity * params.omega / params.charge / params.faradaysConstant
```

## The Numbers

The numbers for the new $i_0$ and $i_1$ given $v_0$=1.65$\times$10$^{-9}$ and $v_1$=3.67$\times$10$^{-12}$

In [5]:
def i_(v, omega=7.1e-6, n=2, F=9.6485e4):
    return v * n * F / omega

print('i0 value:',i_(1.65e-9))
print('i1 value:',i_(3.67e-12))

i0 value: 44.845140845070425
i1 value: 0.09974646478873239
